# Run batch inferncing in local ray cluster

## Step 1: Copy custom image to kind

In [ ]:
import json
from ecs.utils import load_manifest

In [ ]:
#THIS IS THE ONLY CELL YOU NEED TO MODIFY
PROJECT_NAME = 'nba'

In [ ]:
project_dir = f'./projects/{PROJECT_NAME}'

In [ ]:
image_name = "default-image"
manifest = load_manifest(f"{project_dir}/manifest.json")
image_name = manifest.deploy_config.project_name

In [ ]:
image_name

In [ ]:
#copy docker image to local k8s cluster
!kind load docker-image {image_name} --name ray

## Create Deployment File

In [ ]:
!python gendeployfile.py --project {PROJECT_NAME} --env local

## Deploy Ray Cluster

In [ ]:
!kubectl apply -f ray-cluster-deploy.yml

In [ ]:
print(f"run the folowing command in a prompt window: \n kubectl port-forward service/raycluster-{manifest.deploy_config.project_name}-head-svc 8265:8265")

## Submit batch job

In [ ]:
!python batchjob.py --project {PROJECT_NAME} --endpoint http://127.0.0.1:8265  --sourcepath s3://yans-ray-test/{PROJECT_NAME}/test_data.csv --sourceformat csv --resultpath s3://yans-ray-test/{PROJECT_NAME}/batch_output --resultformat csv

## Delete Ray Cluster

In [ ]:
!kubectl delete -f ray-cluster-deploy.yml